# 🚦 Semaphore Detector - Google Colab Server

**Run AI inference with FREE GPU on Google Colab!**

This notebook sets up:
1. FastAPI server for webcam frame processing
2. ngrok tunnel for public access
3. GPU-accelerated inference

---

## 📋 Instructions:
1. **Enable GPU**: Runtime → Change runtime type → GPU
2. **Run all cells** in order
3. **Copy the ngrok URL** and paste it in your frontend config.js

---

## Step 1: Check GPU Status

In [ ]:
# Check if GPU is available
!nvidia-smi

import torch
print(f"\n✅ PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📹 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## Step 2: Install Dependencies

In [ ]:
%%capture
# Install required packages
!pip install fastapi uvicorn python-multipart pyngrok nest_asyncio
!pip install opencv-python-headless numpy pillow
!pip install inference

print("✅ All dependencies installed!")

## Step 3: Configure ngrok

⚠️ **Get your free ngrok authtoken:**
1. Go to [ngrok.com](https://ngrok.com) and sign up (free)
2. Copy your authtoken from [dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)
3. Paste it below

In [ ]:
# PASTE YOUR NGROK AUTHTOKEN HERE
NGROK_AUTHTOKEN = ""  # <-- Paste your token here

if not NGROK_AUTHTOKEN:
    print("⚠️ Please set your ngrok authtoken above!")
    print("📍 Get it free at: https://dashboard.ngrok.com/get-started/your-authtoken")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ ngrok configured!")

## Step 4: Create the FastAPI Server

In [ ]:
import os
import time
import base64
from datetime import datetime
from typing import Optional, Dict, List, Any
from collections import defaultdict

import cv2
import numpy as np
from PIL import Image

from fastapi import FastAPI, WebSocket, WebSocketDisconnect, File, UploadFile, Form, Query
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse

# ============================================================
# CONFIGURATION - Edit these if needed
# ============================================================

API_KEY = "ylFu6Gi5msSoDxbPC9Sl"  # Your Roboflow API key
MODEL_ID = "semaphore-dataset-1wlaa/1"  # Your model ID
CONFIDENCE_THRESHOLD = 0.60

# ============================================================
# LOAD MODEL
# ============================================================

print("🔄 Loading model...")

# Enable GPU for inference
os.environ["ROBOFLOW_INFERENCE_DEVICE"] = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📍 Device: {os.environ['ROBOFLOW_INFERENCE_DEVICE']}")

from inference import get_model
model = get_model(model_id=MODEL_ID, api_key=API_KEY)
print(f"✅ Model loaded: {MODEL_ID}")

# ============================================================
# CREATE FASTAPI APP
# ============================================================

app = FastAPI(title="Semaphore Detector API")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Session storage
session_results: Dict[str, Dict[str, Any]] = defaultdict(dict)

@app.get("/")
async def root():
    return {
        "status": "running",
        "service": "Semaphore Detector API (Colab)",
        "device": os.environ.get("ROBOFLOW_INFERENCE_DEVICE", "unknown"),
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health():
    return {"status": "healthy", "model_loaded": True}

@app.post("/api/process-frame")
async def process_frame(
    file: UploadFile = File(...),
    session_id: Optional[str] = Form(None)
):
    start_time = time.time()
    
    try:
        # Read image
        contents = await file.read()
        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        
        if img is None:
            return {"error": "Invalid image", "detections": []}
        
        # Run inference
        results = model.infer(img)[0]
        predictions = results.predictions
        
        # Format detections
        detections = []
        for pred in predictions:
            if hasattr(pred, 'class_name'):
                confidence = float(pred.confidence)
                if confidence >= CONFIDENCE_THRESHOLD:
                    detections.append({
                        "class": pred.class_name,
                        "confidence": confidence,
                        "x": float(pred.x),
                        "y": float(pred.y),
                        "width": float(pred.width),
                        "height": float(pred.height),
                        "bbox": [
                            float(pred.x - pred.width / 2),
                            float(pred.y - pred.height / 2),
                            float(pred.x + pred.width / 2),
                            float(pred.y + pred.height / 2)
                        ]
                    })
        
        result_data = {
            "detections": detections,
            "timestamp": time.time(),
            "latency": (time.time() - start_time) * 1000
        }
        
        if session_id:
            session_results[session_id] = result_data
        
        return result_data
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return {"error": str(e), "detections": []}

@app.get("/api/latest-results")
async def get_latest_results(session: str = Query(...)):
    if session in session_results:
        return session_results[session]
    return {"detections": [], "message": "No results yet"}

@app.websocket("/ws/stream")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    
    try:
        while True:
            data = await websocket.receive_text()
            start_time = time.time()
            
            try:
                # Parse base64 image
                if ',' in data:
                    header, encoded = data.split(',', 1)
                else:
                    encoded = data
                
                img_data = base64.b64decode(encoded)
                nparr = np.frombuffer(img_data, np.uint8)
                img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                
                if img is None:
                    await websocket.send_json({"error": "Invalid image", "detections": []})
                    continue
                
                # Run inference
                results = model.infer(img)[0]
                predictions = results.predictions
                
                detections = []
                for pred in predictions:
                    if hasattr(pred, 'class_name'):
                        confidence = float(pred.confidence)
                        if confidence >= CONFIDENCE_THRESHOLD:
                            detections.append({
                                "class": pred.class_name,
                                "confidence": confidence,
                                "x": float(pred.x),
                                "y": float(pred.y),
                                "width": float(pred.width),
                                "height": float(pred.height),
                                "bbox": [
                                    float(pred.x - pred.width / 2),
                                    float(pred.y - pred.height / 2),
                                    float(pred.x + pred.width / 2),
                                    float(pred.y + pred.height / 2)
                                ]
                            })
                
                latency = (time.time() - start_time) * 1000
                await websocket.send_json({
                    "detections": detections,
                    "timestamp": time.time(),
                    "latency": latency
                })
                
            except Exception as e:
                await websocket.send_json({"error": str(e), "detections": []})
                
    except WebSocketDisconnect:
        print("📡 Client disconnected")

print("✅ FastAPI app created!")

## Step 5: Start Server with ngrok Tunnel

🚀 **After running this cell, copy the ngrok URL and use it in your frontend!**

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

# Start ngrok tunnel
PORT = 8000
public_url = ngrok.connect(PORT)

print("=" * 60)
print("🚀 SEMAPHORE DETECTOR SERVER IS RUNNING!")
print("=" * 60)
print(f"")
print(f"📍 PUBLIC URL: {public_url}")
print(f"")
print(f"📋 Copy this URL to your frontend config.js:")
print(f"   BACKEND_URL: '{public_url}'")
print(f"   WS_URL: '{str(public_url).replace('https://', 'wss://').replace('http://', 'ws://')}/ws/stream'")
print(f"")
print("=" * 60)
print("⏳ Server is running... Keep this cell running!")
print("   Press STOP to shut down the server.")
print("=" * 60)

# Run server
uvicorn.run(app, host="0.0.0.0", port=PORT)

---

## 🔧 Troubleshooting

### Session Timeout
- Free Colab sessions last up to 12 hours
- GPU sessions may timeout after ~3 hours of inactivity
- Keep the browser tab open and occasionally interact

### ngrok Errors
- Make sure you set your authtoken in Step 3
- Free tier allows 1 active tunnel at a time
- The URL changes each time you restart

### Model Loading Errors
- Check your Roboflow API key is correct
- Verify the model ID exists in your Roboflow account

### Connection Issues
- The public URL may take a few seconds to become active
- Try refreshing your frontend page
- Check if CORS is properly configured